In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow_hub as hub
import tensorflow as tf
import sys
sys.path.append('../')

%load_ext autoreload
%autoreload 2

In [2]:
prefix = ''

In [3]:
from clean_helpers import *

take_full = False
test_locally = True

# Specify here what cleaning functions you want to use
cleaning_actions = ['clean_new_line', 'clean_tags', \
                    'remove_numbers', 'clean_punctuation']


clean = {
    "clean_new_line": clean_new_line,
    "lowercase": lowercase,
    "lemmatize": lemmatize,
    "remove_stopwords": remove_stopwords,
    "translate": perform_translation,
    "clean_punctuation": clean_punctuation_2,
    "clean_tags" : clean_tags,
    "remove_numbers": remove_numbers,
}

In [4]:
if take_full:
    input_file_pos = 'Data/train_pos_full.txt'
    input_file_neg = 'Data/train_neg_full.txt'
else:
    input_file_pos = 'Data/train_pos.txt'
    input_file_neg = 'Data/train_neg.txt'
    
input_file_test = 'Data/test_data.txt'

In [5]:
pos_sentences = []
with open(prefix + input_file_pos, 'r') as f:
    for sentence in f:
        pos_sentences.append(sentence)
        
neg_sentences = []
with open(prefix + input_file_neg, 'r') as f:
    for sentence in f:
        neg_sentences.append(sentence)

In [6]:
pos_data = pd.DataFrame(pos_sentences, columns=['sentence'])
pos_data['label'] = 1
neg_data = pd.DataFrame(neg_sentences, columns=['sentence'])
neg_data['label'] = -1

data = pd.concat([pos_data, neg_data])

In [7]:
for c in cleaning_actions:
    data = clean[c](data)

In [8]:
url = "https://tfhub.dev/google/elmo/2"
embed = hub.Module(url, trainable=True)

RuntimeError: Exporting/importing meta graphs is not supported when eager execution is enabled. No graph exists when eager execution is enabled.

In [ ]:
# This tells the model to run through the 'sentences' list and return the default output (1024 dimension sentence vectors).
embeddings = embed(
    sentences,
    signature="default",
    as_dict=True)["default"]

#Start a session and run ELMo to return the embeddings in variable x
with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  sess.run(tf.tables_initializer())
  x = sess.run(embeddings)